
<center>
    <h1> INF285/ILI285 Computación Científica </h1>
    <h1> Tarea N°1, v1.00 </h1>
    
</center>

<p>

</p>

## Instrucciones

* La tarea es individual.
* Las consultas sobre las tareas se deben realizar por medio de la plataforma Aula.
* La tarea debe ser realizada en `Jupyter Notebook` (`Python3`).
* Se evaluará la correcta utilización de librerias `NumPy`, `SciPy`, entre otras, así como la correcta implementación de algoritmos de forma vectorizada.
*  **El archivo de entrega debe denominarse ROL-tarea-numero.ipynb**. _De no respetarse este formato existirá un descuento de **50 puntos**_
* La fecha de entrega es el jueves 30 de Abril a las **18:00 hrs**.  Se aceptarán entregas hasta las 19:00 hrs sin descuento en caso de existir algun problema, posteriormente existirá un descuento lineal hasta las 20:00 hrs del mismo día.
* Las tareas que sean entregadas antes del jueves a mediodía recibirán una bonificación de 10 puntos
* Debe citar cualquier código ajeno utilizado (incluso si proviene de los Jupyter Notebooks del curso).




In [1]:
# Importamos las librerías que se utilizaran.
import numpy as np
from scipy.special import legendre

## Introducción 

En esta primera tarea de INF/ILI-285, versión 2020-1, estudiaremos la importancia de los primeros temas estudiados en el curso, los cuales son: Representación de Punto Flotante, Pérdida de Significancia, Errores de Cancelación y Búsqueda de Ceros. El desarrollo de cada uno de esos temas se presenta en una serie de preguntas, donde ustedes deben ir decidiendo, pregunta a pregunta, como cada uno de los temas se aplica. En general, los temas no se analizan por separado, sino de manera acoplada. Es muy importante que cada uno de los problemas sea analizado teóricamente primero, para luego poner su conocimiento en acción. Cada problema puede ser desarrollado de diversas formas, sin embargo, es muy importante determinar al final si el camino elegido resuelve la pregunta presentada.

## Problemas

### 1. Simulador (50 ptos) 

Dada la familia de polinomios de grado 3 con la forma:


\begin{equation}
f(x)=x^3 - A\,x^2 + A\,x - 1
\end{equation}


Se pide implementar un algoritmo que reciba como parámetros el valor de $A$ (con $|A|>>1$) y la cantidad de bits que tiene en la mantisa un computador ficticio, el manejo del exponente debe considerarse de las misma forma que lo maneja _double precision_.
Esta implementación debe calcular las raíces de $f$ sin perdida de significancia con la cantidad de bits disponibles para la mantisa.
Para obtener las raíces de $f$ usted debe encontrar de forma algebraica sus raíces y luego proponer un algoritmo basado en las _fórmulas_ obtenidas.

Considere que en ese computador ficticio cuenta con las operaciones matemáticas necesarias para obtener las raíces.  Considere el siguiente ejemplo:

```python
# Alg. Base
a = 9819824.624837
b = 148736.523476
c = a+b

# Alg. con Representación de Punto Flotante de 'bits_mant' bits en la mantisa.
am = f_new_rep(9819824.624837,bits_mant) # Aproximar el input en la nueva representación.
bm = f_new_rep(148736.523476,bits_mant) # Aproximar el input en la nueva representación.
cm = f_new_rep(m,exp,am+bm) # Aproximar el output de la suma y cada operación en la nueva representación.
```


In [2]:
def flToBin(x, bits_mant): # Definimos una función para representar la parte no entera de un numero en base 10 en binario.
    n = float('0.'+ x)
    mant = list()
    while bits_mant > 0:
        if n!= 0:
            n = n*2
            mant.append(int(n))
            if n >= 1:
                n = n-1
        else:
            mant.append(0)
        bits_mant -= 1
    return mant

"""
input
x         : (double) valor a evaluar
bits_mant : (int) cantidad de bits de la mantisa
output
eval      : (double) resultado obtenido
"""
def f_new_rep(x,bits_mant):
    # Algoritmo de representación de punto flotante modificada.
    signo = 0 # El signo se representa como 0 para x>0 y como 1 para x<1
    mantissa = list()
    x = float(x)
    b = '{:f}'.format(x)
    if x < 0: # Modificamos el signo en caso de que el número sea negativo.
        signo = 1
        x *= -1
    int_part = int(x)
    fl_part = str(b).split('.')[1]
    
    if int_part > 0: #Tenemos dos casos, si x > 1 => exponente > 1023
        # El exponente será la cantidad de números (en binario) que hay después del primero 1.
        # Ej: 10101 tiene por exponente 4, que se corresponde al largo de 0101.
        exp = str(bin(int_part)).split("b")[1][1:len(str(bin(int_part)).split("b")[1])]
        # Añadimos los números que siguen del primer 1 a la mantisa.
        for i in range(len(exp)):
            mantissa.append(int(exp[i]))
        # Manteniendo el formato de "IEEE 754 double precision", utilizamos un bias de 1023 para el exponente.
        exp = bin(len(exp) + 1023)
        # Obtenemos la parte decimal del número y lo añadimos a la mantisa respetando su cantidad de dígitos.
        mantissa = mantissa + flToBin(str(fl_part), bits_mant-len(mantissa)) 
    else: # Y cuando x < a 1 => exponente < 1023
        exp = flToBin(str(fl_part), bits_mant)
        cont = 1
        exp_f = 0
        for i in range(0,bits_mant+1): # Contamos los 0's hasta el primer 1 para obtener el exponente.
            if int(exp[i]) == 1: 
                cont = i+1
                break
            exp_f += 1
        # Calculamos la representación binaria del largo de la mantisa más la cantidad de ceros antes del primer 1.
        mantissa = flToBin(str(fl_part), bits_mant+cont)
        # La mantisa finalmente corresponde a los dígitos que están después del primer 1.
        mantissa = mantissa[ mantissa.index(1)+1 : ]
        exp = bin(-exp_f-1 + 1023)
    exp = 2**(int(exp, 2)-1023)
    mant = 0    
    for i in range(len(mantissa)):# Obtenemos la representación en base 10 de la mantisa.
        mant += mantissa[i]*2**(-(i+1)) #(bit)2^-(1+posición del bit)
    # Finalmente la nueva representación viene dada por: ((-1)^signo)*(1+mant)*(2^exp).
    result = (-1)**signo*(mant+1)*exp
    return result

def f_find_roots(A,bits_mant):
    # Algoritmo para encontrar raíces de f(x).
    # Obtenemos la nueva representación para A.
    A = f_new_rep(A, bits_mant)

    # Calculamos sus raíces según la formula obtenida en el despeje.
    x2 = (A+1 + np.sqrt(A**2 - 2*A -3))*0.5
    x3 = (A+1 - np.sqrt(A**2 - 2*A -3))*0.5

    # Obtenemos la nueva representación de cada raíz y retornamos.
    x_roots = [1, f_new_rep(x2, bits_mant), f_new_rep(x3, bits_mant)]
    return x_roots


### Para poder encontrar las raíces de la función primero se factorizo.
\begin{align}
    f(x) &= x^3 - A\,x^2 + A\,x - 1\\
    f(x) &= (x-1)(x^2 - (A-1)x +1)
\end{align}


Como queremos los ceros, entonces buscamos $f(x) = 0$, luego:


\begin{align}
    (x-1)(x^2 - (A-1)x +1) &= 0
\end{align}


De donde se tiene que:


\begin{align}
    x-1 = 0\\
    x^2 - (A-1)x +1 = 0
\end{align}


Obteniendo que:


\begin{align}
        x_1 &= 1\\
        x_2 &= \frac{1+A + \sqrt{A^2-2A-3}}{2}\\
        x_3 &= \frac{1+A - \sqrt{A^2-2A-3}}{2}
\end{align}

### Polinomios de Legendre (50 puntos)
Dada la función compuesta $f$
\begin{equation}
f_{n,m}(x) = L_n(C_m(x)),
\end{equation}
donde $L_n$ es conocido como el polinomio de Legendre de grado $n$ definido de la siguiente forma:
\begin{equation}
L_{n}(x)=\frac{1}{2^{n}} \sum_{k=0}^{n}\left(\begin{array}{l}
n \\
k
\end{array}\right)^{2}(x-1)^{n-k}(x+1)^{k},
\end{equation}
y $C_m$ es el polinomio de Chebyshev
\begin{equation}
C_m(x) = \cos(m \cdot \arccos(x)).
\end{equation}

Utilizando el método de Bisección y Punto fijo se pide obtener la raíz de $f$ más cercana a $0.5$ dado un valor de $m$ y $n$



*Hint: Las raíces de Legendre son conocidas*



##### Para $L_N$ usaremos legendre que se encuentra en scipy y lo evaluaremos con np.polyval de numpy

In [3]:
# Definimos el polinomio de Chebyshev y el de Legendre
cm = lambda x,m: np.cos(m*np.arccos(x))
ln = lambda n: legendre(n)
f = lambda x,n,m: np.polyval(ln(n), cm(x,m))

In [4]:
"""
input
n: (int) grado del polinomio de Legendre
m: (int) grado del polinomio de Chebyshev
tol: (double) tolerancia para la detención del algoritmo
output
root: (double) raiz obtenida
"""
def f_Biseccion(n, m, tol):
    a = 0.3
    b = 0.9
    while (b-a)/2 > tol:
        c = (a+b)/2

        # Evaluamos la función en los puntos a,b y c.
        f_a = f(a,n,m)
        f_b = f(b,n,m)
        f_c = f(c,n,m)
        
        # Actualizamos el valor de a o b según corresponda
        if f_a*f_c<0: 
            b = c
        elif f_c*f_b<0:
            a = c
        # Si no se cumple con los criterios anteriores, entones no se puede encontrar una raíz entre los puntos a y b.
        else: 
            return "NO SE PUEDE ENCONTRAR UNA RAIZ EN EL RANGO [" + str(a) +',' + str(b) + ']'
    root = c
    return root

def f_FPI(n, m, tol):
    x_0 = 0.45 # Definimos nuestro initial guess
    root = f(x_0, n, m) # Calculamos x_1 como el valor obtenido de evaluar la función en x_0
    error = np.inf # Definimos nuestro error como un número muy grande.
    while error > tol: #I teramos hasta cumplir con nuestra tolerancia.
        x_i = root # Nuestro x_i será nuestro último resultado
        root = f(x_i, n, m) # Calculamos x_{i+1}(root)
        error = abs(root-x_0) # Actualizamos el error
    return root


#### En el algoritmo de bisección se escogió a = 0.3 y b= 0.9 ya que 0.5 se encuentra entre estos valores y convergio para distintos n's y m's.

###  Determinantes (20 puntos)

Dada una matriz de dimensiones $ n \times n$ de la forma:
\begin{equation}
A
=
\begin{pmatrix}
a_{1,1} & a_{1,2}  & \dots & a_{1,n} \\
a_{2,1} & a_{2,2}  & \dots & a_{2,n} \\
\vdots &  \vdots   & \ddots & \vdots \\
a_{n,1} & a_{n,2}  & \dots & a_{n,n}
\end{pmatrix}
=
\begin{pmatrix}
\mathbf{r}_1 \\
\mathbf{r}_2 \\
\vdots \\
\mathbf{r}_n \\
\end{pmatrix},
\end{equation}
donde $\mathbf{r}_k$ representa la $k$-ésima fila de la matriz $A$.
Considere la siguiente matriz $C_{i,j}(\alpha)$,
\begin{equation}
C_{i,j}(\alpha)
=
\begin{pmatrix}
\mathbf{r}_1 \\
\vdots \\
\mathbf{r}_i\,(1-\alpha)+\mathbf{r}_j\,\alpha \\
\vdots \\
\mathbf{r}_j\,(1-\alpha)+\mathbf{r}_i\,\alpha \\
\vdots \\
\mathbf{r}_n \\
\end{pmatrix},
\end{equation}
de lo cual sabemos que $C_{i,j}(0)=A$ y que $C_{i,j}(1)$ es la matriz $A$ donde se intercambiaron las filas $i$ y $j$, es decir:
\begin{equation}
C_{i,j}(1)
=
\begin{pmatrix}
\mathbf{r}_1 \\
\vdots \\
\mathbf{r}_j \\
\vdots \\
\mathbf{r}_i \\
\vdots \\
\mathbf{r}_n \\
\end{pmatrix}.
\end{equation}
De las relaciones anteriores podemos concluir que el determinante de la matriz $A$, denominado $D=\det(A)$, es igual al determinante de $C_{i,j}(0)$, es decir $\det(C_{i,j}(0))=\det(A)=D$.
Por el otro lado, el determinante de $C_{i,j}(1)$ es $-D$, dado que es el intercambio de las filas $i$ y $j$ de la matriz $A$.
Por lo cual podemos concluir que $-D\leq \det(C_{i,j}(\alpha))\leq D$.

Utilizando el método de Bisección debe encontrar el valor de $\alpha$ con $p$ decimales de precisión que permitan que, dado los índices de las filas $i$, $j$, y $i\neq j$, el determinante de la matriz sea igual a $d$, donde $d\in[-D,D]$. 

Para esto se debe implementar una función que reciba la matriz $A$, las filas $i$ y $j$, y $p$; y retorne $\widehat{\alpha}$ tal que $\det(C_{i,j}(\widehat{\alpha}))=d$.


In [5]:
"""
input
A: (array n x n) matriz
i: (int) índice de la fila "i".
j: (int) índice de la fila "j".
p: (int) cantidad de decimales de precision 
d: (double) valor requerido del determinante de $C_{i,j}(\alpha)$, i.e. $\det(C_{i,j}(\widehat{\alpha}))=d$.
output
alpha_hat: (double) alpha_hat tal que det(C_{i,j}(alpha_hat))=d.
"""
def find_alpha_hat(A,i,j,p,d):
  # Algoritmo a desarrollar, note que usted debe definir el intervalo a utilizar en el método de la bisección.
  return alpha_hat

# Referencias

[1] Claudio Torres et al, Apuntes de Computación Científica,2020